## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,44.89,97,100,8.97,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.51,93,75,6.91,light rain
2,2,Icod De Los Vinos,ES,28.3724,-16.7119,90.54,66,100,9.40,overcast clouds
3,3,Vestmannaeyjar,IS,63.4427,-20.2734,43.99,70,63,7.20,broken clouds
4,4,Albany,US,42.6001,-73.9662,38.91,68,52,5.01,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Atuona,PF,-9.8000,-139.0333,79.05,77,48,16.33,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,77.99,66,4,11.86,clear sky
9,9,Hermanus,ZA,-34.4187,19.2345,70.81,52,12,16.17,few clouds
11,11,Puerto Ayora,EC,-0.7393,-90.3518,75.16,85,95,7.54,overcast clouds
14,14,Kahului,US,20.8947,-156.4700,73.85,79,20,18.01,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                269
City                   269
Country                266
Lat                    269
Lng                    269
Max Temp               269
Humidity               269
Cloudiness             269
Wind Speed             269
Current Description    269
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                266
City                   266
Country                266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Current Description    266
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Atuona,PF,79.05,scattered clouds,-9.8000,-139.0333,
6,Rikitea,PF,77.99,clear sky,-23.1203,-134.9692,
9,Hermanus,ZA,70.81,few clouds,-34.4187,19.2345,
11,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
14,Kahului,US,73.85,few clouds,20.8947,-156.4700,
16,Sabang,ID,84.07,broken clouds,5.8933,95.3214,
20,Arraial Do Cabo,BR,71.46,broken clouds,-22.9661,-42.0278,
21,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
27,Umm Lajj,SA,83.59,overcast clouds,25.0213,37.2685,
29,Mahibadhoo,MV,84.76,overcast clouds,3.7833,72.9667,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    #get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping") 
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")

hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df.count()

City                   249
Country                249
Max Temp               249
Current Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [13]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Atuona,PF,79.05,scattered clouds,-9.8000,-139.0333,Villa Enata
6,Rikitea,PF,77.99,clear sky,-23.1203,-134.9692,People ThankYou
9,Hermanus,ZA,70.81,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
11,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Kahului,US,73.85,few clouds,20.8947,-156.4700,Maui Seaside Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))